<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

In [83]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import folium
from bs4 import BeautifulSoup
import requests
import geocoder # import geocoder
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


## 1. Download and Explore Dataset

In [40]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[0]

df_temp = pd.read_html(str(table))

df = pd.DataFrame(df_temp[0])

In [41]:
print(df.shape)
df.head()

(180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Deleting the not assigned Borough's row

In [42]:
df = df[df.Borough != 'Not assigned'].reset_index(drop=True)
print(df.shape)
df[0:12]

(103, 3)


,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [34]:
Lat_list=[]
Lng_list=[]

for i in range(df.shape[0]):
    address='{}, Toronto, Ontario'.format(df.at[i,'Postal Code'])
    g = geocoder.arcgis(address)
    Lat_list.append(g.latlng[0])
    Lng_list.append(g.latlng[1])

In [43]:
df['Latitude'] = Lat_list
df['Longitude'] = Lng_list
df[0:12]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667481,-79.528953
6,M1B,Scarborough,"Malvern, Rouge",43.808626,-79.189913
7,M3B,North York,Don Mills,43.748900,-79.357220
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.707193,-79.311529
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.377529


In [44]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [47]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='orange',
        fill=True,
        fill_color='#3006cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [48]:
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

In [49]:
CLIENT_ID = 'EI1RT5DKEAPLTXBX3YWBFPYKX5O034BYAPI1TFZBCQWEEYVD' # your Foursquare ID
CLIENT_SECRET = 'UD1WQYEQKENGAIM3IZC0ALCAMFKBQPTB1AQOF0E5HVTERN0A' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EI1RT5DKEAPLTXBX3YWBFPYKX5O034BYAPI1TFZBCQWEEYVD
CLIENT_SECRET:UD1WQYEQKENGAIM3IZC0ALCAMFKBQPTB1AQOF0E5HVTERN0A


#### Exploring the first neighborhood

In [63]:
df.loc[0, 'Neighborhood']

'Parkwoods'

In [69]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    df.loc[0, 'Latitude'], 
    df.loc[0, 'Longitude'], 
    radius, 
    LIMIT)

results = requests.get(url).json()

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-69-b7d7dde53ee4>:30: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Brookbanks Park,Park,43.751976,-79.332140
1,Variety Store,Food & Drink Shop,43.751974,-79.333114
2,649 Variety,Convenience Store,43.754513,-79.331942


## 2 - Exploring all neighborhoods in Toronto

In [70]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [71]:
toronto_venues = getNearbyVenues(names=df['Neighborhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )



Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [74]:
print(toronto_venues.shape)
toronto_venues.head()

(2274, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.752935,-79.335641,649 Variety,43.754513,-79.331942,Convenience Store
3,Victoria Village,43.728102,-79.311890,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.728102,-79.311890,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [79]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",19,19,19,19,19,19
Bayview Village,2,2,2,2,2,2
"Bedford Park, Lawrence Manor East",20,20,20,20,20,20
Berczy Park,63,63,63,63,63,63
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",44,44,44,44,44,44
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100


In [80]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 265 uniques categories.


## 3 - Analysis

In [81]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
              venue  freq
0    Breakfast Spot   0.2
1   Badminton Court   0.2
2       Supermarket   0.2
3  Sushi Restaurant   0.2
4      Skating Rink   0.2


----Alderwood, Long Branch----
               venue  freq
0  Convenience Store  0.11
1           Pharmacy  0.11
2     Sandwich Place  0.11
3        Gas Station  0.11
4                Gym  0.11


----Bathurst Manor, Wilson Heights, Downsview North----
                 venue  freq
0                 Bank  0.11
1          Coffee Shop  0.11
2  Fried Chicken Joint  0.05
3             Pharmacy  0.05
4        Deli / Bodega  0.05


----Bayview Village----
                        venue  freq
0  Construction & Landscaping   0.5
1                       Trail   0.5
2                 Yoga Studio   0.0
3         Moroccan Restaurant   0.0
4     New American Restaurant   0.0


----Bedford Park, Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.10
1         Coffee Shop  0.10
2  Italian Restaurant  0.10
3

                 venue  freq
0          Coffee Shop  0.07
1                Hotel  0.05
2  Japanese Restaurant  0.03
3           Restaurant  0.03
4                 Café  0.03


----Studio District----
                 venue  freq
0       Baseball Field   0.2
1   Athletics & Sports   0.2
2  Government Building   0.2
3         Night Market   0.2
4     Business Service   0.2


----Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park----
                venue  freq
0  Light Rail Station  0.33
1         Coffee Shop  0.22
2                Park  0.11
3        Liquor Store  0.11
4         Supermarket  0.11


----The Annex, North Midtown, Yorkville----
               venue  freq
0     Sandwich Place  0.11
1               Café  0.11
2        Coffee Shop  0.07
3  Indian Restaurant  0.04
4      Grocery Store  0.04


----The Beaches----
               venue  freq
0              Trail   0.2
1             Church   0.2
2  Health Food Store   0.2
3                Pub   0.2
4        Yoga Stu

In [114]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Breakfast Spot,Badminton Court,Skating Rink,Supermarket,Sushi Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Convenience Store,Athletics & Sports,Gas Station,Pub
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Diner,Restaurant
3,Bayview Village,Construction & Landscaping,Trail,Women's Store,Ethiopian Restaurant,Dog Run
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Pizza Place,Thai Restaurant


## 4 Clustering

In [115]:
# set number of clusters
kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 0, 3, 3, 3, 3, 3, 3], dtype=int32)

In [116]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [117]:
toronto_merged[10:21] # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,M6B,North York,Glencairn,43.707279,-79.447500,3.0,Pizza Place,Japanese Restaurant,Pub,Fast Food Restaurant,Sushi Restaurant
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650023,-79.554089,3.0,Pizza Place,Tea Room,Chinese Restaurant,Sandwich Place,Dumpling Restaurant
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.785779,-79.157368,NaN,NaN,NaN,NaN,NaN,NaN
13,M3C,North York,Don Mills,43.722143,-79.352023,3.0,Athletics & Sports,Trail,Other Great Outdoors,Restaurant,Coffee Shop
14,M4C,East York,Woodbine Heights,43.689740,-79.308507,3.0,Grocery Store,Pharmacy,Coffee Shop,Smoke Shop,Pet Store
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,3.0,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub
16,M6C,York,Humewood-Cedarvale,43.691730,-79.430013,1.0,Grocery Store,Park,Hockey Arena,Field,Trail
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.637813,-79.576484,3.0,Pizza Place,Pet Store,Convenience Store,Park,Cosmetics Shop
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,3.0,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Beer Store
19,M4E,East Toronto,The Beaches,43.678148,-79.295349,3.0,Health Food Store,Church,Pub,Trail,Women's Store


In [118]:
toronto_merged.columns = [c.replace(' ', '_') for c in toronto_merged.columns]
toronto_merged = toronto_merged[toronto_merged['Cluster_Labels'].notna()]
toronto_merged[10:21] 

,Postal_Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue
10,M6B,North York,Glencairn,43.707279,-79.447500,3.0,Pizza Place,Japanese Restaurant,Pub,Fast Food Restaurant,Sushi Restaurant
11,M9B,Etobicoke,"West Deane Park, Princess Gardens, Martin Grov...",43.650023,-79.554089,3.0,Pizza Place,Tea Room,Chinese Restaurant,Sandwich Place,Dumpling Restaurant
13,M3C,North York,Don Mills,43.722143,-79.352023,3.0,Athletics & Sports,Trail,Other Great Outdoors,Restaurant,Coffee Shop
14,M4C,East York,Woodbine Heights,43.689740,-79.308507,3.0,Grocery Store,Pharmacy,Coffee Shop,Smoke Shop,Pet Store
15,M5C,Downtown Toronto,St. James Town,43.651734,-79.375554,3.0,Café,Coffee Shop,Cocktail Bar,Cosmetics Shop,Gastropub
16,M6C,York,Humewood-Cedarvale,43.691730,-79.430013,1.0,Grocery Store,Park,Hockey Arena,Field,Trail
17,M9C,Etobicoke,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",43.637813,-79.576484,3.0,Pizza Place,Pet Store,Convenience Store,Park,Cosmetics Shop
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765806,-79.185284,3.0,Fast Food Restaurant,Pizza Place,Restaurant,Coffee Shop,Beer Store
19,M4E,East Toronto,The Beaches,43.678148,-79.295349,3.0,Health Food Store,Church,Pub,Trail,Women's Store
20,M5E,Downtown Toronto,Berczy Park,43.645196,-79.373855,3.0,Coffee Shop,Cocktail Bar,Breakfast Spot,Seafood Restaurant,Hotel


In [120]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Analysing the clusters

#### Cluster 1

In [121]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue
6,Scarborough,0.0,Trail,Women's Store,Electronics Store,Doctor's Office,Dog Run
26,Scarborough,0.0,Construction & Landscaping,Trail,Women's Store,Ethiopian Restaurant,Dog Run
39,North York,0.0,Construction & Landscaping,Trail,Women's Store,Ethiopian Restaurant,Dog Run


#### Cluster 2

In [122]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue
0,North York,1.0,Food & Drink Shop,Convenience Store,Park,Women's Store,Ethiopian Restaurant
5,Etobicoke,1.0,Park,Baseball Field,Skating Rink,Women's Store,Ethiopian Restaurant
16,York,1.0,Grocery Store,Park,Hockey Arena,Field,Trail
21,York,1.0,Park,Women's Store,Gym,Mexican Restaurant,Spa
27,North York,1.0,Park,Dog Run,Residential Building (Apartment / Condo),Electronics Store,Doctor's Office
31,West Toronto,1.0,Park,Grocery Store,Pet Store,Pool,Smoke Shop
35,East York,1.0,Music Venue,Women's Store,Distribution Center,Fish & Chips Shop,Field
36,Downtown Toronto,1.0,Theme Park,Park,Harbor / Marina,Farm,Electronics Store
41,East Toronto,1.0,Discount Store,Park,Business Service,Bus Line,Grocery Store
49,North York,1.0,Park,Basketball Court,Bakery,Ethiopian Restaurant,Dog Run


#### Cluster 3

In [123]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue
68,Central Toronto,2.0,Gym / Fitness Center,Ethiopian Restaurant,Doctor's Office,Dog Run,Donut Shop


#### Cluster 4

In [124]:
toronto_merged.loc[toronto_merged['Cluster_Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster_Labels,1st_Most_Common_Venue,2nd_Most_Common_Venue,3rd_Most_Common_Venue,4th_Most_Common_Venue,5th_Most_Common_Venue
1,North York,3.0,Pizza Place,Portuguese Restaurant,Intersection,French Restaurant,Park
2,Downtown Toronto,3.0,Pub,Café,Coffee Shop,Athletics & Sports,Performing Arts Venue
3,North York,3.0,Clothing Store,Toy / Game Store,Bookstore,Furniture / Home Store,Cosmetics Shop
4,Downtown Toronto,3.0,Coffee Shop,Café,Hobby Shop,Distribution Center,Sandwich Place
7,North York,3.0,Athletics & Sports,Trail,Other Great Outdoors,Restaurant,Coffee Shop
8,East York,3.0,Pizza Place,Fast Food Restaurant,Pharmacy,Athletics & Sports,Breakfast Spot
9,Downtown Toronto,3.0,Coffee Shop,Clothing Store,Middle Eastern Restaurant,Sandwich Place,Italian Restaurant
10,North York,3.0,Pizza Place,Japanese Restaurant,Pub,Fast Food Restaurant,Sushi Restaurant
11,Etobicoke,3.0,Pizza Place,Tea Room,Chinese Restaurant,Sandwich Place,Dumpling Restaurant
13,North York,3.0,Athletics & Sports,Trail,Other Great Outdoors,Restaurant,Coffee Shop


## 5 - Conclusion

The conclusion we can make is that :
- Cluster 1 contains any kind of construction and landscraping.
- Cluster 2 gots any relaxing venue such as parks yoga studios and paygrounds.
- Cluster 3 can be considered as an outlier with gyms and fitness centers as a common venues.
- And cluster 4 regroup mostly food shops and restaurent.